In [12]:
# !pip install glob3
# !pip install Pillow
!pip install torchvision

  Using cached torchvision-0.6.1-cp36-cp36m-manylinux1_x86_64.whl (6.6 MB)
     |###                             | 76.6 MB 1.1 MB/s eta 0:10:36^C

ERROR: Operation cancelled by user


In [11]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split




import PIL

import glob
import os

from collections import defaultdict
import random


    

In [2]:
def read_files(data_path):
    work_dir = os.getcwd()
    os.chdir(data_path)
    data_dict = defaultdict(dict)
    
    try:
        for file in glob.glob('./*/*'):
            path_split = file.split('/')[1:]
            data_dict[path_split[0]][path_split[1]] = os.listdir(file)
    except Exception as e:
        print(e)
        os.chdir(work_dir)
    os.chdir(work_dir)
    
    return data_dict

In [3]:
train_dir = 'data/omniglot/images_background/'
train_dict = read_files(train_dir)

test_dir = 'data/omniglot/images_evaluation/'
test_dict = read_files(test_dir)

In [4]:
class ImageLoader(Dataset):
    def __init__(self, data_path, data_size):
        self.data_path = data_path
        self.data_size = data_size
        self.data_dict = self.read_files()
        self.category = list(self.data_dict.keys())
    
    def __len__(self):
        return self.data_size
    
    def read_files(self):
        work_dir = os.getcwd()
        os.chdir(self.data_path)
        data_dict = defaultdict(dict)

        try:
            for file in glob.glob('./*/*'):
                path_split = file.split('/')[1:]
                data_dict[path_split[0]][path_split[1]] = os.listdir(file)
        except Exception as e:
            print(e)
            os.chdir(work_dir)
        os.chdir(work_dir)

        return data_dict
    
    def sample_images(self, same_class=True):
        img1 = img2 = class_ = None
        cat = random.choice(self.category)
        if same_class:
            char = random.choice(list(self.data_dict[cat].keys()))
            img1, img2 = random.sample(self.data_dict[cat][char], 2)
            
            img1 = ('/').join([cat, char, img1])
            img2 = ('/').join([cat, char, img2])
            return [img1, img2, 1]
        
        if random.random() > 0.5:
            char1, char2 = random.sample(list(self.data_dict[cat].keys()), 2)
            img1 = random.choice(self.data_dict[cat][char1])
            img2 = random.choice(self.data_dict[cat][char2])

            img1 = ('/').join([cat, char1, img1])
            img2 = ('/').join([cat, char2, img2])
        else:
            char1 = random.choice(list(self.data_dict[cat].keys()))
            img1 = random.choice(self.data_dict[cat][char1])
            img1 = ('/').join([cat, char1, img1])

            cat2 = random.choice(self.category)
            char2 = random.choice(list(self.data_dict[cat2].keys()))
            img2 = random.choice(self.data_dict[cat2][char2])
            img2 = ('/').join([cat2, char2, img2])
        
        return [img1, img2, 0]   
              
        
    
    def __getitem__(self, index):
        if index%2==0:
            return self.sample_images(same_class=True)
        return self.sample_images(same_class=False)   
        

In [6]:
imgLoader = ImageLoader(train_dir, 10000)
train_set, val_set = random_split(imgLoader, [8000, 2000])